# Anchors

**Anchors는 예측을 충분히 '고정'시키는 의사결정 규칙을 찾아 블랙박스 분류 모델에 대한 개별 예측을 설명하는 방법이다.** 여기서 규칙은 다른 설명변수 값의 변화가 예측에 미치는 영향을 미치지 않는 경우 예측을 고정시킨다. Anchor는 그래프 검색 알고리즘과 함께 강화 학습 기법을 활용하여 모델 호출 횟수(필요한 실행시간)를 최소로 줄이는 방식이며, LIME을 도입한 **Ribeiro, Singh, Guestrin**이 제안한 알고리즘이다. LIME과 마찬가지로 Anchor는 perturbation을 사용하여 블랙박스 모형의 예측에 대한 Local Explainer이다. 하지만 LIME이 사용하는 대리모형(Surrogate Model) 대신 결과에 대한 해석은 Anchor라 불리는 이해하기 쉬운 IF-THEN 규칙으로 표현된다.  
어떤 표본에 있는 10개의 설명변수가 $x=(x_1,x_2,...,x_{10})$으로 관측되었을 때, 분류를 목적으로 하는 블랙박스 모형 $f$에 $x$를 입력하여 $f(x)$가 범주 0으로 예측했다고 가정하자. $f(z_1,x_2,x_3,z_4,x_5,z_6,...,z_{10})$을 앞의 표본에서는 $x_2,x_3,x_5$는 그대로 유지하고 나머지 설명변수 값은 임의로 $z_1,z_4,z_6,...,z_{10}$으로 바꿔서 입력했을 때의 블랙박스 예측치라고 하자.  
예를 들어, $z_1,z_4,z_6,...,z_{10}$을 100번 바꿔서 입력했을 때 이 중 98번은 class 0으로 예측했다면, 설명변수 $x_2,x_3,x_5$는 블랙박스 모형에서 범주 0으로 예측하는데 중요한 역할을 하는 반면, $z_1,z_4,z_6,...,z_{10}$에 대응하는 나머지 설명변수 $x_1,x_4,x_6,...,x_{10}$은 class 0으로 예측하는데 거의 기여하지 않는다고 할 수 있다.  
위의 예시에서 class 0으로 예측하는데 중요한 역할을 하는 변수들인 $x_2,x_3,x_5$를 **anchors** 라고 말하며 A={$x_2,x_3,x_5$}로 표기한다. 만약 A를 포함하고 있는 표본 수가 전체 학습데이터의 30%를 차지하고 있다면 anchors A의 **coverage** 는 30%가 된다고 할 수 있다. 이럴 경우 anchors A는 높은 신뢰성과 일반화가 가능한 anchors라고 할 수 있다.  
Anchors는 이처럼 **분류**를 목적으로 하는 블랙박스 모형의 설명 가능성을 핵심 설명변수로 표현하여 사람이 쉽게 이해할수 있도록 제안되었다. 회귀 Task에 대해서는 지원하지 않는다.

## Anchors의 결정
Anchors A를 결정하는 절차를 설명한다.  
$(1)$. If-Then 룰을 anchors라고 한다.  
$(2)$. anchors가 주어졌을 때 anchors 이외의 설명변수 값을 다른 값으로 바꿔도 예측에는 거의 변함이 없다.  
$(3)$. 즉, anchors를 포함하고 있는 표본들의 예측은 거의 같다고 할 수 있다.  

특정 표본 $x='This\ movie\ is\ not\ bad'$을 입력하면 블랙박스 모형 $f(x)='positive'$로 예측되었고, $A$={$'not','bad'$}라고 가정하자. $D$를 설명변수 $x$의 분포로 표기하고, $D(\bullet |A)$를 anchors $A$가 주어졌을 때의 분포라고 가정한다. 다음 표는 $D$와 $D(\bullet |A)$의 예를 보여준다.

<center>

| 분포            | 표본                                                                                                               | 비고                                       |
| :--:           |:------------------------------------------------------------------------------------------------------------------:|:-----------------------------------------:|
|  $D$           | This director is always bad<br>This movie is not nice<br>This stuff is rather honest<br>This star is not bad<br>...| 학습데이터                                  |
| $D(\bullet\|A)$ | This audio is not bad<br>This novel is not bad<br>This football is not bad                                         | 학습데이터 중 $A=${$'not'$,$'bad'$}를 가진 표본들 |

</center>

위의 표는 설명변수 $x$의 분포 $D$와 이 분포로부터의 표본을 보여준다. $D(\bullet |A)$는 anchors $A$가 주어졌을 때의 설명변수 $x$의 조건부 분포이다. $D$에서의 표본 $x$가 anchors $A$를 포함하면 $A(x)=1$로 정의한다. 그러면 설명변수의 부분집합 $A$가
$$
A(x)=1\ 이고\ E_z(1_{[f(x)=f(Z)]}|A)\ge \tau
$$
를 만족하면 $A$를 anchors라고 말한다. 여기에서 대문자 $Z$는 관측치 $z$에 대응하는 확률변수이고 $f$는 예측모형이다.

$$
E_z(1_{[f(x)=f(Z)]}|A)=P(f(x)=f(Z)|A) \tag 1
$$
이므로 1에 가까운 값을 갖는 $\tau$에 대해

$$
P(f(x)=f(Z)|A)\ge \tau  
$$
의 의미를 표본 관점에서 해석하면, $\tau=0.98$일 때 $D(\bullet |A)$에 있는 표본 100개를 뽑아 $f(z)$를 구하면 이 중 98개 이상은 $f(z)=f(x)$라는 의미이다. 즉, 설명변수 값 $x$을 갖는 예측치 범주인 $f(x)$와 이 설명변수 값 $x$ 중 $A$에 속한 설명변수를 고정한 설명변수의 집합인 $D(\bullet |A)$에서 임의로 뽑은 설명변수 $z$의 범주 $f(z)$도 98% 이상 원래의 범주 $f(x)$와 일치하면 이 $A$를 anchors라고 한다.

식 $(1)$에 정의된 $P(f(x)=f(Z)|A)$를 $prec(A)$로 표기할 때, anchors의 조건을 다음과 같이 완화한다.
$$
P(prec(A)\ge \tau)\ge \ 1-\delta \tag 2
$$

여기서 $\delta$ 를 혀용오차라고 한다. 표본의 관점에서 설명하면, 예를 들어 $\tau=0.98$이고, $\delta =0.01$이라고 가정하면 $D(\bullet|A)$로부터 표본 $z$를 100개 뽑아 $f(z)=f(x)$인 표본 개수가 98개 이상이면 $\tau=0.98$을 만족하고, 이러한 절차를 100번 반복하면 1번 정도는 $\tau=0.98$을 만족하지 못해도 된다는 의미이다. $\delta$는 최소의 표본으로 anchors A를 결정하는 **MAB(Multi-Armed-Bandit)** 알고리즘의 상한, 하한 신뢰구간을 구하는데 사용된다. MAB 알고리즘은 뒤에서 설명한다.

식 $(2)$의 허용오차 $\delta$ 때문에 $P(f(x)=f(Z)|A)\ \ge \tau$를 만족하는 2개 이상의 anchors A가 존재할 수 있다. 예를 들어 $A=\left\{ x_1,x_2 \right\}$, $A'=\left\{ x_2,x_4 \right\}$가 모두 anchor의 조건을 만족한다고 가정하자. 만약 전체 데이터 ($D$)에서 $A$를 포함하고 있는 표본 수가 $A'$을 포함하고 있는 표본 수보다 많다면 anchors $A$의 일반화가 더 우수하므로 $A$를 선택하게 될 것이다. 이를 **coverage**라고 한다.

$$
coverage(A)=E_D(A(X)) \tag 3
$$
으로 coverage는 정의된다.

$D$에 있는 표본 $x$가 $A$를 포함하고 있으면 $A(x)=1$이므로
$$
E_D(A(X))=P(A\in X)
$$
가 된다. 여기서 $X$는 $x$에 대응하는 확률변수이다. 그러므로 **전체 표본에서 anchors $A$를 포함하고 있는 설명변수를 가진 표본 수의 비율이 $coverage(A)$의 추정치가 된다.** 지금까지 논의한 anchors $A$를 정리하면 식 $(2)$를 만족하는 anchors 중 가장 큰 $coverage(A)$를 가진 anchors가 anchors $A$이다. 수학적 표현으로 anchor $A$는 다음과 같이 표현한다.

$$
\max_{A \text{ s.t. } P(\text{prec}(A) \geq \tau) \geq 1 - \delta} \text{coverage}(A) \tag 4
$$

특정 표본의 설명변수 $x$에서 식 $(4)$를 만족하는 $A$를 greedy하게 구하는 것은 많은 연산을 요구하기 때문에 거의 불가능하다. 이러한 이유로 수많은 슬롯머신에서 가장 수익이 좋은 m개의 슬롯머신을 최소의 시행으로 찾아내는 **Multi-Armed-Bandit problem**에서 정의한 **KL-LUCB** 알고리즘을 이용한다.

좀 많이 복잡하여 다시 한번 간단하게 정리하고자 한다.  
먼저, **Predicate**란, 하나의 설명변수에 대한 술어이다. 수치형 변수에 대해선, $29 \le\ age\ \le 34$와 같이 표현되며, 범주형 변수에 대해선, `sex=male`처럼 표현할 수 있다. 이러한 predicate가 **AND**로 연결되어 있는 것을 **rule**이라고 한다. 예시로, [$29 \le\ age\ \le 34$ **AND** `sex=male` **AND** `Job= teacher`]가 하나의 Rule이 될 수 있다.  
**Anchor explanation**이란 **블랙박스 모형의 local prediction을 충분히 고정(anchors)시켜주는 Rule이다.** Rule 이외의 변수의 변화는 예측에 영향을 미치지 못한다. **Anchor의 목표는 Precision이라는 제약 조건하에서 Coverage를 최대화하는 Anchor를 찾는 것이다.**  
- **Coverage** : Anchor를 포함하는 표본의 비율  
- **Precision** : Anchor의 규칙을 고정하고 나머지 변수이 변경되었을 때, 블랙박스 모델이 기존의 분류 결과와 동일한 결과를 주는 표본의 비율

---

### MAB : Multi-Armed-Bandit

책에는 MAB에 대한 구체적인 설명이 없지만, Anchors 뿐만 아니라 강화학습,인과추론에서도 사용되며, **실제 비즈니스 어플리케이션에서 상당한 성과를 거두고 있는 알고리즘**이기 때문에 조금 더 구체적으로 설명을 한다.

#### Motivation
옛날에, 슬롯머신으로 엄청나게 흥행을 한 카지노가 있다. 이 카지노에는 전 세계의 다양한 슬롯머신들이 있다. 그러다가 카지노의 고객들은 유독 **특정 슬롯머신의 수익률이 좋다는 사실**을 경험적으로 알게 되었다.

<div style="text-align: center;">
    <img src="https://media.npr.org/assets/img/2022/07/29/ap21188557258852_wide-7b9355bc9f3f776844d5ceb5148fe6d60c40eccd.jpg?s=1100&c=85&f=webp" alt="Example Image" width="700">
    <p style="font-style: italic; text-align: center;">출처: https://media.npr.org/assets/img/2022/07/29/ap21188557258852_wide-7b9355bc9f3f776844d5ceb5148fe6d60c40eccd.jpg?s=1100&c=85&f=webp</p>
</div>

어느 날, 수학자 한 명이 카지노에 방문을 했다. 이 수학자는 문득 **"어떤 방식으로 슬롯머신에 투자를 하면 최적의 수익을 얻을 수 있을까?"** 에 대해 생각하게 된다.  
N개의 슬롯머신이 있다. 각각의 슬롯머신은 수익률이 다르다. 그런데 당장 각 슬롯머신의 수익률을 알지는 못한다. 여기서 내 돈을 어느 슬롯머신에 걸고 슬롯머신의 암(손잡이)를 내려야 돈을 제일 많이 벌 수 있을까? 카지노에는 여러 개의 슬롯머신이 있기 때문에, 이 문제의 이름이 **Multi-Armed-Bandits**이다.

#### 탐색과 활용 (Exploration & Exploitation)

MAB의 핵심 문제는 **탐색과 활용**이다. 예를 들어서 오늘 카지노에 방문했을 때, 3가지 슬롯머신 기계를 동시에 플레이한다. 이제 돈을 벌기 위한 전략을 수립한다.

- ***전략 1: Greedy***  
**한 번씩 플레이 후, 가장 높은 점수를 주는 슬롯 머신에 몰빵**  
각 슬롯머신을 한 번씩 플레이 후, 돈을 가장 많이 딴 슬롯머신에 모두 투자한다. 3대의 슬롯머신 중에 다음과 같은 결과가 나왔다고 하자.  
<center>

|     슬롯머신    |   수익  |
| :-----------: | :----:  | 
| 슬롯머신1(Arm)  | 1000원  |
| 슬롯머신2(Arm)  |   0원   |
| 슬롯머신3(Arm)  |  500원  |

</center>

<div style="text-align: center;">
    <p style="font-style: italic; text-align: center;">"슬롯머신 1번이 가장 높은 수익을 주니, 슬롯머신 1에 투자한다."</p>
</div>

상식적으로 생각하면, 각 슬롯머신별로 한 번만 테스트를 진행하여 신뢰할 수 없는 전략인 것을 알 수 있다. **탐색(Exploration)이 충분히 이루어지지 않았다.** 기본적으로 슬롯머신은 확률 게임인데, 단 한 번의 플레이 만으로 슬롯머신의 가치를 판단하는 것 자체가 문제이다. 탐색을 충분히 하지 않았기에, 이 전략은 좋지 않다.  
이 전략을 바로 **greedy 알고리즘**이라고 한다.  
$$
Q_t(a)\doteq \frac{sum\ of\ rewards\ when\ a\ taken\ prior\ to\ t}{\#\ of\ times\ a\ taken\ prior\ to\ t}
$$

<div style="text-align: center;">
    <p style="font-style: italic; text-align: center;">이 슬롯머신으로 얻은 보상의 합/전체 슬롯머신 시도 횟수</p>
</div>

$$
A_t \doteq \arg\max_{a} Q_t(a)
$$

<div style="text-align: center;">
    <p style="font-style: italic; text-align: center;">기대보상을 최대로 만들어주는 슬롯머신 선택</p>
</div>

여기서 $Q_t(a)$는 행동 a의 평균 보상을 의미한다.

- ***전략 2: epsilon-greedy***  
**동전을 던져서 윗면이 나오면 점수가 좋았던 슬롯머신, 뒷면이 나오면 랜덤 선택**  
탐색이 부족했던 전략 1을 수정한 전략이다. 먼저 일정한 확률로 랜덤으로 슬롯머신을 선택하도록 한다. 동전의 앞면이 나올 확률은 50%이다. 50%의 확률로 greedy 알고리즘으로 경험상 좋았던 슬롯머신을 선택하고, 나머지 50%의 확률로 동전 뒷면이 나오면 슬롯머신의 성능(보상)에 상관없이 랜덤으로 슬롯머신을 고른다.  
이 알고리즘을 **epsilon-greedy(e-greedy)** 알고리즘이라고 부른다. 여기서 동전의 앞면이 나올 50%의 확률이 epsilon이라는 **하이퍼파라미터**이다.

- ***전략 3: UCB(Upper-Confidence-Bound)***  
**좋은 수익률을 보이며 최적의 선택이 될 가능성이 있는 슬롯머신을 선택**  
전략 2는 최적의 슬롯머신을 찾기 위해 랜덤으로 탐색을 한다. 하지만 꼭 랜덤이 좋은 방식인지에 대한 의문이 생긴다. 숨겨준 최적의 선택을 찾기 위해선 **탐색**을 해야 한다. 슬롯머신이 최적이 될 수 있을만한 가능성을 수치로 계산해서 가장 가능성이 있는 슬롯머신을 선택하면 어떨까?  
<div style="text-align: center;">
    <p style="font-style: italic; text-align: center;">"아직 좋은 성능을 보여주진 못하지만, 더욱 시도를 해보면 성능이 좋아질 것 같다."</p>
</div>  
  
UCB 알고리즘에서 다음 행동을 선택하는 알고리즘은 다음과 같다.
$$
A_t \doteq \arg\max_{a} \left[ Q_t(a) + c \sqrt{\frac{\log t}{N_t(a)}} \right]
$$
$N_t(a)$는 해당 슬롯머신을 선택했던 횟수이다. $c$는 탐색의 정도를 조절할 수 있는 하이퍼파라미터이며, $t$는 모든 슬롯머신을 선택한 횟수의 합이다. UCB 알고리즘을 전략 1과 비교해본다.
$$
A_t \doteq \arg\max_{a} Q_t(a)
$$
UCB 알고리즘에서 추가된 부분은 **해당 슬롯머신이 최적의 슬롯머신이 될 수 있는 가능성 혹은 불확실성**을 의미한다. 해당 식에서 UCB 알고리즘에서 추가된 부분을 빼면 **LCB(Lower-Confidence-Bound)** 가 된다.

이외에도 다양한 알고리즘이 존재하고, Anchors에서 사용하는 KL-LUCB 또한 MAB 문제에서 사용할 수 있는 알고리즘 중 하나이다. 이름에서 추측할 수 있듯이, KL-Divergence를 응용한 알고리즘이다.

---

## Anchors A를 결정하는 절차

Anchor를 찾고 결정하는 방법은 매우 복잡하므로 한 번에 이해하기 힘들 것이다. 먼저 우리가 설명하고 싶은 특정 표본의 설명변수를 $x=\left\{ x_1,x_2,...,x_M \right\}$으로 표기한다. anchors $A$를 결정하는 절차는 anchors 후보 선택과 최적의 anchors $A$를 결정하는 두 개의 절차로 나눌 수 있다. Iterative 방식으로 후보 선택과 결정을 하며, $t=1$(라운드 수)부터 시작하여 다음 절차$(1)$과 절차$(2)$를 반복한다.

<div style="text-align: center;">
    <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fl8eZI%2FbtqGJvSAgHd%2FACqICpRuEtq6B8kaAKUTzk%2Fimg.jpg" alt="Example Image" width="700">
</div>

### 절차(1): Anchors 후보 선택  
(1). 첫 번째 Iteration이면($t=1$), $\left\{ x_1,x_2,...,x_M \right\}$에서 $coverage(x_i) \gt c$를 만족하는 $\Lambda_1 \gets \Lambda_1\cup x_1$이다.  

(2). $t \gt 1$이면 모든 $A \in \Lambda_{t-1}$에 대해 유일한 $\left\{ A\cup x_l,A \cup x_{l+1},...,A\cup x_L\right\}$ 집합을 구성한다. 여기서, $x_l,x_{l+1},...,x_L \in \left\{ x_1,x_2,..,x_M \right\}$이지만 $\not \in \Lambda_{t-1}$이다.  

(3). 만약 $m=l,l+1,...,L$에 대해 $coverage(A \cup x_m) \gt c$이면 $\Lambda_t\gets \Lambda \cup x_m$이다.

### 절차(2): Anchors A 결정  
(1). 각 $A \in \Lambda_t$에 대해 $D(\bullet|A)$로부터 $n$개의 표본을 뽑아 $Prec(A),Prec_{lwr}(A),Prec_{upr}(A)$를 구한다.


$$
Prec(A) = P(f(x)=f(Z)|A)
$$
$$
Prec_{lwr}=Prec(A)- \sqrt{\frac{1}{2N_A(t)}ln(\frac{5/4\ \bullet s_tt^4}{\delta})}
$$
$$
Prec_{upr}=Prec(A)+ \sqrt{\frac{1}{2N_A(t)}ln(\frac{5/4\ \bullet s_tt^4}{\delta})}
$$
여기서 $s_t$는 $\Lambda$의 크기이며, $N_A(t)$는 $t-1$ 시점까지의 $A$를 포함한 누적 표본 수이다.

(2).  
$$
A= \arg\max_{A}Prec(A)
$$
$$
A'= \arg\max_{A'}Prec(A')
$$
$A' \neq A$으로 정의한다.

(3). $Prec_{upr}(A')-Prec_{lwr}(A) \gt \epsilon$이면 $Prec_{upr}(A')-Prec_{lwr}(A) \le \epsilon$이 될 때까지 다음을 반복한다.

(3-1). $D(\bullet|A)$와 $D(\bullet|A')$로부터 각각 표본을 뽑아(즉, $N_A(t)와 N_{A'}(t)$를 증가시켜) $A$와 $A'$ 각각에 대한 $Prec,Prec_{lwr},Prec_{upr}$를 update한다.  

(3-2). 만약 $Prec_{upr}(A')-Prec_{lwr}(A) \le \epsilon$이지만 $Prec_{lwr}(A) \le \tau$이면 $t=t+1$으로 증가시키고, (1)번 step과 (3)번 step으로 돌아간다. 만약 $Prec_{upr}(A')-Prec_{lwr}(A) \le \epsilon$이고, $Prec_{lwr}(A) \gt \tau$이면 $A$를 anchors로 결정하고 알고리즘을 stop한다.

#### Anchor 알고리즘 예시

상당히 복잡하므로, 구체적인 예시를 통해 이해하자. $x=(x_1,x_2,x_3)$로, 3개의 설명변수를 가진 데이터셋의 표본이다. 먼저 절차(1)의 $\Lambda_1$의 예비후보는 $\left\{x_1  \right\},\left\{x_2  \right\},\left\{x_3  \right\}$이고 $c=0.05$일 때, $coverage(x_1),coverage(x_3) \gt c$이고, $coverage(x_2) \lt c$이면 $\Lambda_1=\left\{ x_1,x_3 \right\}$이 된다.  

절차(2)에서는 $\Lambda_1$를 받아서 $X_1=x_1$으로 관측된 표본 중, 임의로 n개를 뽑는다. 만약 $n=10$이면 설명변수 $(x_1,z_2,z_3)$를 가진 표본 10개를 뽑아  $f(x_1,x_2,x_3)=f(x_1,z_2,z_3)$인 비율로 $Prec(x_1)$을 추정하고 동일한 방법으로 $Prec(x_3)$를 추정한다. 즉, $D(z|x_1)$으로부터 10개의 표본을 추출하여 이 중 7개가 $f(x_1,x_2,x_3)$와 같은 예측 범주이면 $Prec(x_1)=0.7$이고, $D(z|x_3)$으로부터 10개의 표본을 추출하여 이 중 5개가 $f(x_1,x_2,x_3)$의 예측 범주와 같다면 $Prec(x_3)=0.5$가 된다.  
그러므로, $\Lambda_1=\left\{ x_1,x_3 \right\}$에서 $A=x_1$이며, $A'=x_3$이 된다. $N_{x_1}(1)=N_{x_3}(1)=0$이므로 절차(2)의 (3-1)을 실행한다. 만약 $\epsilon=0.01$이고 $\tau=0.98$라고 하면, $Prec_{upr}(x_3)-Prec_{lwr}(x_1) \le 0.01$이지만 $Prec_{lwr}(x_1) \le 0.98$이면 절차(1)의 (3)번 Step을 반복하게 된다.  
  
$\Lambda_2$의 예비후보는 $\left\{ (x_1,x_2),(x_1,x_3),(x_3,x_2) \right\}$이 된다. 이 예비 후보의 coverage가 0.05를 넘으면 $Prec(x_1,x_2),Prec(x_1,x_3),Prec(x_2,x_3)$를 구해서 가장 큰 $Prec$을 가진 설명변수로 $A$를 정의하고 절차(2)의 (3-2) step의 stop조건을 만족하면 이 $A$를 anchors로 결정하고 알고리즘을 종료한다. $t$를 증가시켜 시도하지 않고 종료하는 이유는 $A$의 크기가 작을수록 anchors의 설명 가능성이 높아지기 때문이다.

---

절차(2)에서 표본추출할 때, $D(\bullet|A)$에서 임의 추출하는데, Anchors의 표본추출은 tabular,text,image 데이터에 따라 다르게 정의된다.

- Tabular  
Tabular 데이터에서 먼저 수치형 변수일 경우 먼저 **범주형으로 변환**하여야 하며 일반적으로 분위수(quantile)로 연속형 설명변수를 범주화(discretization)하지만 도메인 지식에 기반하여 범주화할 수도 있다. 범주화된 설명변수로 재정의된 학습데이터로부터 $D(\bullet|A)$를 구성하고 이로부터 표본을 임의추출하게 된다.  
- Text  
Text 데이터는 Tabular 데이터와 다르게, 선택된 표본으로 직접 $D(\bullet|A)$를 구성하게 된다. 이는 LIME과 동일한 표본 생성 개념이다. 즉 anchors가 아닌 말뭉치는 \<unk\>로 대체하거나 가장 유사한 말뭉치(embedding 값의 측면에서 내적합이 가장 큰), 또는 BERT와 같은 사전학습 모형을 이용하여 예측한 단어로 대체하여 $D(\bullet|A)$를 생성한다.  
- Image  
Image 데이터의 경우, anchors에 속하지 않는 super-pixel은 표본이미지의 평균으로 대체하거나 특정 색(보통 검은색 또는 흰색)으로 대체하여 $D(\bullet|A)$를 생성한다.

---

## Anchors의 적용과 응용
앞에서 논의했듯이 $D(\bullet|A)$의 표본추출 방법이 데이터에 형태에 따라 다르므로 `alibi`에서는 `AnchorTabular`,`AnchorText`,`AnchorImage`로 나누어 데이터의 형태에 따라 anchors를 구하도록 한다.

### Tabular 데이터
Tabular 데이터에서 anchors를 추출하는 방법을 논의하기 위해 필요한 라이브러리를 호출하고 `alibi`로부터 `AnchorTabular`를 호출한다. fetch_adult라는 tabular 데이터를 alibi.datasets에서 호출한다.

In [1]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder

from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult

fetch_adult 데이터의 목적값은 연수입이 5만 불 이상인지 아닌지의 이진분류 값을 가지고 있으며 설명변수는 인구통계학적 변수로 실수값을 갖는 설명변수와 범주형 설명변수로 구성되어 있다. 아래 adult.key()에서 category_map은 범주형 설명변수를 순서형 정수로 전환하고 범주형 자료의 이름을 설명변수명_범주명을 변환한 범주형 설명변수의 이름이다. one-hot-encoding된 범주형 설명변수명을 부여하여 anchors를 출력할 때 설명 가능성을 높이기 위함이다.

In [2]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [3]:
data=adult.data
target=adult.target

feature_names=adult.feature_names
category_map=adult.category_map

print(category_map.keys())
print(category_map)

dict_keys([1, 2, 3, 4, 5, 6, 7, 11])
{1: ['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'], 2: ['Associates', 'Bachelors', 'Doctorate', 'Dropout', 'High School grad', 'Masters', 'Prof-School'], 3: ['Married', 'Never-Married', 'Separated', 'Widowed'], 4: ['?', 'Admin', 'Blue-Collar', 'Military', 'Other', 'Professional', 'Sales', 'Service', 'White-Collar'], 5: ['Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'], 6: ['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'], 7: ['Female', 'Male'], 11: ['?', 'British-Commonwealth', 'China', 'Euro_1', 'Euro_2', 'Latin-America', 'Other', 'SE-Asia', 'South-America', 'United-States', 'Yugoslavia']}


다음과 같이 설명변수(data)와 목적값을 shuffling 한 후, 설명변수 data와 목적값인 target을 정의하고 있다. 최초 30000개의 표본을 학습데이터로 정의하고 나머지 데이터를 시험 데이터로 분리한다.

In [5]:
from alibi.utils import gen_category_map

In [6]:
np.random.seed(0)

data_perm = np.random.permutation(np.c_[data,target])
data=data_perm[:,:-1]
target=data_perm[:,-1]

idx=30000
X_train,y_train = data[:idx,:],target[:idx]
X_test,y_test = data[idx:,:],target[idx:]

다음은 범주형 설명변수를 제외한 실수형 설명변수만 추려서 missing 값을 해당 설명변수의 중위수로 채워준 후(impute), 실수형 설명변수를 모두 표준화하도록 한다.

In [9]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])

범주형 설명변수를 모아서 missing이 발생한 설명변수는 자신의 중위수로 대체한 후, one-hot encoding으로 변환한다.

In [10]:
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('onehot',OneHotEncoder(handle_unknown='ignore'))])

다음과 같이 `ColumnTransformer`를 이용하여 앞에서 정의한 두 개의 Pipeline을 실행하고 X_train 학습데이터에 적합한다. 아래의 Preprocessor는 anchors를 구하기 위한 예측함수를 정의할 때 사용한다. 원래의 입력 설명변수를 preprocessor에 통과시켜 anchors가 `ColumnTransformer` 이전 본래의 설명변수로 구해질 수 있도록 하기 위함이다.

In [16]:
preprocessor = ColumnTransformer(transformers=[('num',ordinal_transformer,ordinal_features),('cat',categorical_transformer,categorical_features)])
preprocessor.fit(X_train);

`ColumnTransformer`로 변환시킨 학습데이터를 random forest 모형에 입력하여 학습시킨다.

In [19]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(preprocessor.transform(X_train),y_train);

predict_fn = lambda x:clf.predict(preprocessor.transform(x))
print('Train accuracy:',accuracy_score(y_train,predict_fn(X_train)))
print('Test accuracy:',accuracy_score(y_test,predict_fn(X_test)))

Train accuracy: 0.9655333333333334
Test accuracy: 0.8559156579461148


분석데이터가 tabular 데이터이므로 `AnchorTabular`를 호출하고 설명변수와 범주형 설명변수의 범주 이름을 지정해준다. 설명변수가 실수형일 때, 실수형 데이터는 범주화한다. anchor의 설명 가능성을 높혀주기 위함이다. 실수형 설명변수는 다음과 같이 25%,50%,75% percentile로 각 실수형 설명변수를 범주화하고 있으며 이러한 범주화 값을 학습데이터 X_train으로부터 구하고 있다.

In [20]:
explainer = AnchorTabular(predict_fn,feature_names,categorical_names=category_map,seed=1)
explainer.fit(X_train,disc_perc=[25,50,75])

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': 1, 'disc_perc': [25, 50, 75]},
  'version': '0.9.6'}
)

x_test의 첫 번째 표본을 입력하여 예측 label을 구하면 연수입이 5만불 이하 label로 예측하고 있다.

In [22]:
idx = 0
class_names = adult.target_names

cls = explainer.predictor(X_test[idx].reshape(1,-1))

print(cls)
print('Prediction:',class_names[cls[0]])

[0]
Prediction: <=50K


Thresold는 식 $(2)$의 $\tau$로 anchors가 주어졌을 때 95% 이상이 X_test[0]의 예측 label(<=50K)과 일치하도록 조건을 부여한다.

In [27]:
explanation = explainer.explain(X_test[idx],threshold=0.95)

print('Anchor:%s'%(' AND ').join(explanation.anchor))
print('Precision: %.2f'%explanation.precision)
print('Coverage: %.2f'%explanation.coverage)

Anchor:Marital Status = Separated AND Education = High School grad
Precision: 0.95
Coverage: 0.10


결과를 살펴보면, anchors는 Marital Status = Seperated AND Education = High School grad로 나타났다. 즉 이혼한 고졸이면 95%정도가 연소득이 5만불 이하의 label이고, coverage는 11%임을 알 수 있다.

다음은 X_test의 7번째 표본에 적용하여 95%이상의 precision을 갖는 anchors를 구한다.

In [78]:
idx=6
class_names = adult.target_names

In [79]:
print('Prediction:',class_names[explainer.predictor(X_test[idx].reshape(1,-1))[0]])
explanation = explainer.explain(X_test[idx],threshold=0.95)

Prediction: <=50K


In [80]:
print('Anchor:%s'%(' AND '.join(explanation.anchor)))

Anchor:Hours per week <= 40.00 AND Capital Gain <= 0.00 AND Workclass = State-gov AND Race = White AND Capital Loss <= 0.00 AND Country = United-States AND Sex = Male AND Age > 47.00 AND Occupation = Professional AND Education = Bachelors


In [81]:
print('Precision: %.2f'%explanation.precision)
print('Coverage: %.2f'%explanation.coverage)

Precision: 0.99
Coverage: 0.00


하지만 다음 결과를 보면 Anchor가 매우 많으며, 정밀도가 1에 가까운 반면, coverage는 0임을 알 수 있다. 이는 주어진 제약 조건인 $\tau=0.95$를 만족하는 단순한 rule이 없다는 것으로 해석할 수 있다. 이러한 결과는 신뢰할 수 없어 보인다.

### Text 데이터
text 데이터로부터 anchors를 구하는 예제를 살펴보기 위해 아래와 같이 필요한 클래스와 라이브러리를 호출한다. `spacy`는 사전학습된 `spaCy`언어 모델을 호출한 후, `AnchorText`에 입력하여 off된 spaCy 언어 모뎋의 예측치로 대체하기 위함이다.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' ## 트랜스포머의 결과 출력 무시

import spacy
import string

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from alibi.explainers import AnchorText
from alibi.datasets import fetch_movie_sentiment
from alibi.utils import DistilbertBaseUncased,BertBaseUncased,RobertaBase

로지스틱 회귀모형을 적합하고 `AnchorText`에 적용할 데이터를 movies로 객체화 한다.

In [3]:
movies = fetch_movie_sentiment()
movies.keys()

dict_keys(['data', 'target', 'target_names'])

설명변수를 data로, 목적변수를 labels로 정의하고 목적변수의 범주명을 taget_names로 정의한다.

In [4]:
data= movies.data
labels=movies.target

target_names = movies.target_names
print(target_names)

['negative', 'positive']


데이터를 학습데이터,검증데이터,시험데이터로 분리하고 movie 데이터의 type이 list이므로 세 개의 데이터를 np.array를 이용하여 텐서데이터로 전환한다.

In [5]:
train,test,train_labels,test_labels = train_test_split(data,labels,test_size=0.2,random_state=42)

train,val,train_labels,val_labels=train_test_split(train,train_labels,test_size=0.1,random_state=42)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

학습 text 자료를 빈도수로 전환하고 이 데이터를 로지스틱 회귀모형에 적합시킨다.

In [6]:
vectorizer =CountVectorizer()
vectorizer.fit(train);

np.random.seed(0)
clf=LogisticRegression(solver='liblinear')
clf.fit(vectorizer.transform(train),train_labels);

예측함수를 정의하고 모형의 성능을 측정하고 `AnchorText`의 예측 모형으로 사용한다. 여기서 $x$는 test 데이터이다. anchors를 입력된 text 자료로 표현해야 설명 가능하기 때문이다.

In [7]:
predict_fn = lambda x:clf.predict(vectorizer.transform(x))

In [8]:
pred_train=predict_fn(train)
pred_val=predict_fn(val)
pred_test=predict_fn(test)

print('Train accuarcy: %.3f'%accuracy_score(train_labels,pred_train))
print('validation accuarcy: %.3f'%accuracy_score(val_labels,pred_val))
print('Test accuarcy: %.3f'%accuracy_score(test_labels,pred_test))


Train accuarcy: 0.980
validation accuarcy: 0.754
Test accuarcy: 0.759


사전학습된 spacy를 nlp로 정의한다.

In [10]:
nlp=spacy.load('en_core_web_md')

다섯 번째 표본을 선택하여 이를 출력하고, 해당 표본 text를 입력한 예측 결과는 'negative'인 것을 볼 수 있다.

In [13]:
text =data[4]
print('* Text: %s' % text)
pred = target_names[predict_fn([text])[0]]
alternative = target_names[1-predict_fn([text])[0]]

print('* Prediction: %s'% pred)

* Text: a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
* Prediction: negative


앞에서 정의한 예측함수 predict_fn을 `AnchorText`에 지정하고 off된 말뭉치를 unknown으로 할 때는 `sampling_strategy='unknown'`으로 지정하고 , `similarity` 일 때는 spacy 사전학습 모델을 지정해야 한다. `sampling_strategy='unknown'`이면 off된 말뭉치는 'UNK'로 'similarity'이면 사전학습 모형인 spacy에서 예측된 말뭉치로 대체하게 된다.

In [15]:
explainer=AnchorText(predictor=predict_fn,sampling_strategy='unknown',nlp=nlp)
explanation=explainer.explain(text,threshold=0.95)

다음은 예측 class가 'negative'로 되게 한 anchors가 'flashy'이고 precision은 0.99로 나타나 'flashy'거 'negative' 범주를 예측하는데 핵심적인 단어임을 보여준다. anchors 'flashy'를 가지고 있고 'negative' 범주로 예측된 대표적인 표본을 보여준다. `sampling_strategy='unknown'`이므로 off된 단어들이 'UNK'로 표시된 것을 볼 수 있다.  
'flash'라는 anchors를 가지고 있으나 'positive' 범주로 원래 표본데이터와 다르게 예측된 대표적 표본도 출력하여 보여주고 있다.

In [23]:
print('Anchor:%s'%(' AND '.join(explanation.anchor)))
print('Precision:%.2f'%explanation.precision)
print('Coverage:%.2f'%explanation.coverage)

print('\nExamples where anchor applies and model predicts %s:'% pred)
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:'% alternative)
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']]))

Anchor:flashy
Precision:0.99
Coverage:0.50

Examples where anchor applies and model predicts negative:
a UNK flashy UNK UNK opaque and emotionally vapid exercise in style UNK mystification .
a UNK flashy UNK UNK UNK and emotionally UNK exercise UNK UNK and UNK UNK
a UNK flashy UNK narratively opaque UNK UNK UNK exercise in style and UNK UNK
UNK visually flashy UNK narratively UNK and emotionally UNK UNK UNK UNK UNK mystification .
UNK UNK flashy UNK UNK opaque and emotionally UNK UNK in UNK and UNK .
a visually flashy but UNK UNK and UNK UNK UNK in style UNK mystification .
a visually flashy but UNK opaque UNK emotionally vapid UNK in UNK and mystification .
a UNK flashy but narratively UNK UNK emotionally vapid exercise in style UNK mystification UNK
a UNK flashy but narratively opaque UNK emotionally vapid exercise in style and mystification .
a visually flashy UNK UNK opaque UNK UNK UNK exercise in UNK UNK UNK .

Examples where anchor applies and model predicts positive:
UNK UNK fla

다음은 `sampling_strategy='similarity'`로 변경하여 off된 단어를 유사한 단어로 바꾸게 하고 `sampling_proba=0.5`로 지정하여 단어를 off 시키는 비율을 50%로 하도록 조절한다.

In [25]:
explainer = AnchorText(predictor=predict_fn,sampling_strategy='similarity',nlp=nlp,sample_proba=0.5)

다음과 같이 볼 수 있듯이 'UNK' 대신 유사한 단어로 교체된 것을 볼 수 있다.

In [26]:
explanation = explainer.explain(text,threshold=0.95)

In [27]:
print('Anchor:%s'%(' AND '.join(explanation.anchor)))
print('Precision:%.2f'%explanation.precision)
print('Coverage:%.2f'%explanation.coverage)

print('\nExamples where anchor applies and model predicts %s:'% pred)
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']]))
print('\nExamples where anchor applies and model predicts %s:'% alternative)
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']]))

Anchor:exercise AND vapid
Precision:0.99
Coverage:0.25

Examples where anchor applies and model predicts negative:
that visually flashy but tragically opaque and emotionally vapid exercise under genre and glorification .
another provably flashy but hysterically bulky and emotionally vapid exercise arounds style and authorization .
that- visually flashy but narratively opaque and politically vapid exercise in style and mystification .
a unintentionally decal but narratively thick and emotionally vapid exercise in unflattering and mystification .
the purposely flashy but narratively rosy and emotionally vapid exercise in style and mystification .
thievery intentionally flashy but hysterically gray and anally vapid exercise in style and mystification .
a irrationally flashy but narratively smoothness and purposefully vapid exercise near style and diction .
a medio flashy but narratively blue and economically vapid exercise since style and intuition .
a visually flashy but narratively opaq

---